In [1]:
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display
import yfinance as yf
import dash
from dash import dcc
from dash import html
import matplotlib.pyplot as plt

In [2]:
df_data = pd.read_excel('data_norm.xlsx', header=[0,1], index_col=0)

## Generating a data sample of 10 randomly drawn stocks

In [3]:
data_sample = df_data.sample(n=10, random_state=3)
data_sample

Tickers       ESG       Environmental       Social       Governance   
Ticker  ESG Score Environmental Score Social Score Governance Score   
TYL      0.590361            0.647482     0.326705         0.581633  \
AIZ      0.484509            0.561151     0.318182         0.459184   
PPL      0.422547            0.438849     0.332386         0.475510   
IPG      0.484509            0.561151     0.318182         0.459184   
NVDA     0.402754            0.428777     0.338068         0.438776   
KMB      0.530120            0.618705     0.326705         0.479592   
PLD      0.545611            0.640288     0.321023         0.489796   
SYK      0.471601            0.525180     0.332386         0.469388   
TROW     0.501721            0.568345     0.332386         0.479592   
GS       0.521515            0.618705     0.326705         0.459184   

Tickers           Adj Close                                           
Ticker  2009-12-31 00:00:00 2010-12-31 00:00:00 2011-12-31 00:00:00   
TYL               19.910000           20.760000           30.110001  \
AIZ               22.574564           30.047615           32.637608   
PPL               15.829987           13.589239           15.975157   
IPG                5.057050            7.277219            6.833824   
NVDA               4.285175            3.532747            3.179471   
KMB               38.814289           40.076164           48.754013   
PLD               16.945395           21.924681           20.518147   
SYK               42.324940           45.667580           42.871140   
TROW              35.513306           43.943409           39.631302   
GS               136.436630          137.176636           74.627296   

Tickers                                                              ...   
Ticker  2012-12-31 00:00:00 2013-12-31 00:00:00 2014-12-31 00:00:00  ...   
TYL               48.439999          102.129997          109.440002  ...  \
AIZ               28.212191           54.971760           57.597473  ...   
PPL               16.351688           18.043226           22.763865  ...   
IPG                7.915779           12.974970           15.533352  ...   
NVDA               2.830577            3.779495            4.814917  ...   
KMB               58.077232           74.283867           88.328667  ...   
PLD               27.059484           28.221949           33.944744  ...   
SYK               48.061153           66.946388           85.295998  ...   
TROW              47.045284           61.756729           64.663788  ...   
GS               107.009567          150.662613          166.911987  ...   

Tickers     Adj Close Daily                                           
Ticker  2022-12-16 00:00:00 2022-12-19 00:00:00 2022-12-20 00:00:00   
TYL              320.299988          320.049988          319.250000  \
AIZ              122.945061          122.278717          121.383636   
PPL               28.871065           28.851244           28.900799   
IPG               32.180096           31.892593           32.199921   
NVDA             165.681870          162.512390          160.822693   
KMB              134.030212          134.366959          134.089630   
PLD              111.426323          111.257561          111.366760   
SYK              238.452988          237.647659          237.220169   
TROW             109.255608          108.276939          108.059456   
GS               343.887695          342.160034          342.040894   

Tickers                                                               
Ticker  2022-12-21 00:00:00 2022-12-22 00:00:00 2022-12-23 00:00:00   
TYL              329.619995          321.829987          320.119995  \
AIZ              122.855553          122.298615          124.347359   
PPL               29.138666           28.920622           29.277422   
IPG               32.616302           32.527081           33.131817   
NVDA             164.981979          153.363953          152.034180   
KMB              135.654572    

## Calculating log returns of the data sample

In [4]:
from log_return import log_returns
log_re = log_returns(data_sample['Adj Close'])

## Fetching ESG scores from the data sample for later use

In [5]:
from fetch_score import get_scores
esg_np, e_np, s_np, g_np = get_scores(data_sample)

## Calculating the Covariance matrix, inverse covariance matrix and mean return for the stocks

In [6]:
from covariance import cov_matrix
from mean_ret import mean_return

cov_var = cov_matrix(log_re)
inv_cov = np.linalg.inv(cov_var)
mean_re = mean_return(log_re)

## Finding two optimal portfolios with different expected return

In [7]:
from ones import vector_ones
from mvp import mvp_func
from weights_mvp import mvp_weights

vector_one = vector_ones(data_sample)
a, b, c, d = mvp_func(inv_cov, mean_re, vector_one)
w1, w2 = mvp_weights(a, b, c, d, inv_cov, vector_one, mean_re, 0.10, 0.26)

## Two optimal portfolios are now obtained. Thus, we can use the two fund theorem to find the weights for the entire efficient frontier. Hereby, we also compute the portfolios expected returns, volatility and weighted avereage esg score

In [8]:
from alpha import alpha
from two_fun_theorem import w3

alpha_list = alpha([-3])
ef_weights, r, vol, esg_wa = w3(w1, w2, alpha_list, mean_re, cov_var, esg_np)

### Tickerlist for the plot

In [9]:
tickers_list = []
labels = data_sample.index
for i in range(len(ef_weights)):
    #assets = np.random.choice(list(labels), len(labels), replace=False)
    assets = list(labels)
    tickers_list.append(assets)
sr_t_list = list(labels)

## Obtaining the global MVP. Furthermore, we calculate the expected return, volatility and WA ESG score for the portfolio

In [10]:
from global_mvp import global_mvp
mvp_weight, mvp_er, mvp_vol, mvp_esg_score, mvp_e_score, mvp_s_score, mvp_g_score = global_mvp(ef_weights, vol, mean_re, cov_var, esg_np, e_np, s_np, g_np)

In [11]:
mvp_er

0.09520000000000312

In [12]:
mvp_vol

0.1061380775721038

In [13]:
from esg_indicators import indicators
indicators(vector_one, inv_cov, mean_re, esg_np)

(0.17954829309931958, 0.48373112204321067)

In [14]:
from esg_investor_weights import calculate_esg_sensitive_investor

In [15]:
list_target_returns = np.arange(r.min(), r.max(), 0.0016).tolist()
esg_investor_weights, esg_portfolio_returns, esg_portfolio_vol, esg_investor_wa  = calculate_esg_sensitive_investor(mean_re, cov_var, esg_np, 0.56, list_target_returns, mvp_weight)

## Sharpe Ratio

### Sharpe ratio for insensitive investor

In [16]:
from calculating_max_sr import calculate_max_sharpe_ratio
sr, w_opt, sharpe_exp, sharpe_vol, sr_esg_score, sr_e_score, sr_s_score, sr_g_score = calculate_max_sharpe_ratio(0.03, mean_re, cov_var, esg_np, e_np, s_np, g_np, mvp_weight, log_re.columns.tolist())

### Sharpe ratio for sensitive investor

In [17]:
from calculating_max_ESG_sr import calculate_max_ESG_sharpe_ratio
sr_esg, w_opt_esg, sharpe_exp_esg, sharpe_vol_esg, sr_esg_esg_score, sr_esg_e_score, sr_esg_s_score, sr_esg_g_score = calculate_max_ESG_sharpe_ratio(0.03, mean_re, cov_var, esg_np, e_np, s_np, g_np, mvp_weight, log_re.columns.tolist(), 0.56)

## Sortino Ratio

### For insensitive investor

In [18]:
from calculating_max_sortino import calculate_sortino_ratio
optimal_weights, optimal_sortino_ratio, sortino_exp, sortino_vol, sortino_score, sortino_e_score, sortino_s_score, sortino_g_score = calculate_sortino_ratio(log_re, 0.03, log_re.columns.tolist(), mvp_weight, mean_re, cov_var, esg_np, e_np, s_np, g_np)

In [19]:
sortino_exp

0.6119923076923075

In [20]:
sortino_vol

0.6865223754559893

In [21]:
optimal_sortino_ratio

2.2548295485925953

In [22]:
from calculate_max_ESG_sortino import calculate_ESG_sortino_ratio
optimal_weights_esg, optimal_sortino_ratio_esg, sortino_exp_esg, sortino_vol_esg, sortino_score_esg, sortino_e_score_esg, sortino_s_score_esg, sortino_g_score_esg = calculate_ESG_sortino_ratio(log_re, 0.03, log_re.columns.tolist(), mvp_weight, mean_re, cov_var, esg_np, e_np, s_np, g_np, 0.56)

## Plotting the Efficient Frontiers

In [23]:
from plot_ef import ef_plot
plot = ef_plot(tickers_list, sr_t_list, 
               ef_weights, vol, r, esg_wa, 
               esg_investor_weights, esg_portfolio_vol, esg_portfolio_returns, esg_investor_wa, 
               w_opt, sharpe_vol, sharpe_exp, sr, sr_esg_score,
               w_opt_esg, sharpe_vol_esg, sharpe_exp_esg, sr_esg, sr_esg_esg_score,
               mvp_weight, mvp_vol, mvp_er, mvp_esg_score, 
               optimal_weights, sortino_vol, sortino_exp, optimal_sortino_ratio, sortino_score, 
               optimal_weights_esg, sortino_vol_esg, sortino_exp_esg, optimal_sortino_ratio_esg, sortino_score_esg,
               ratio='Sharpe Ratio')

plot.show()

In [24]:
benchmark = pd.read_excel("sp500_bench.xlsx", header=[0], index_col=0)

In [25]:
from log_return import log_returns
log_market_returns = log_returns(benchmark)

## Treynor measure for The optimal Sharpe Ratio

### Insensitive investor

In [26]:
from treynor import calculate_treynor
jensens_alpha, beta, timing = calculate_treynor(12, log_re, w_opt, log_market_returns, 0.03)

### Sensitive Investor

In [27]:
alpha_esg, beta_esg, timing_esg = calculate_treynor(12, log_re, w_opt_esg, log_market_returns, 0.03)

## Treynor measure for the optimal Sortino Ratio

### Insensitive investor

In [28]:
alpha_sortino, beta_sortino, timing_sortino = calculate_treynor(12, log_re, optimal_weights, log_market_returns, 0.03)

### Sensitive investor

In [29]:
alpha_esg_sortino, beta_esg_sortino, timing_esg_sortino = calculate_treynor(12, log_re, optimal_weights_esg, log_market_returns, 0.03)